### Test Reseach question using 1000 sentences of the swiss essays

In [1]:
import os
from nltk import tokenize
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
import pandas as pd
from sentence_transformers import SentenceTransformer,util
import re


##############################################################################################################


model = SentenceTransformer('all-mpnet-base-v2', device="cuda")


##############################################################################################################


listStructureWords =  []

dfSrtucturingWords = pd.read_excel("../structureWords.xlsx")

listStructureWords = dfSrtucturingWords.iloc[:,0].tolist()

regex = r"\b(?:{})\b".format("|".join(listStructureWords))
sentences = []

df_fileName = pd.DataFrame({})

for filename in os.listdir(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\only100-swiss-teachers"):
   with open(os.path.join(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\only100-swiss-teachers", filename)) as f:
       text = f.read()
       text = text.replace("ï»¿","")
       sents = re.split(regex, text)
       sents = tokenize.sent_tokenize(text)
       for s in sents:
           sentses = tokenize.sent_tokenize(s)
           if (s.isspace() or len(s) ==0):
               continue
           s = s.lower()
           if (len(s.split()) <5):
               continue
           for ss in sentses:
                 if (len(ss.split()) >5):
                     sentences.append(ss)

##############################################################################################################

corpus_embeddings = model.encode(sentences, show_progress_bar =True, device="cuda")

def mbkmeans_clusters(X, k, mb=500, print_silhouette_values=False):
    """Generate clusters.

    Args:
        X: Matrix of features.
        k: Number of clusters.
        mb: Size of mini-batches. Defaults to 500.
        print_silhouette_values: Print silhouette values per cluster.

    Returns:
        Trained clustering model and labels based on X.
    """
    km = MiniBatchKMeans(n_clusters=k, batch_size=mb).fit(X)
    print(f"For n_clusters = {k}")
    print(f"Silhouette coefficient: {silhouette_score(X, km.labels_):0.2f}")
    print(f"Inertia:{km.inertia_}")

    if print_silhouette_values:
        sample_silhouette_values = silhouette_samples(X, km.labels_)
        print(f"Silhouette values:")
        silhouette_values = []
        for i in range(k):
            cluster_silhouette_values = sample_silhouette_values[km.labels_ == i]
            silhouette_values.append(
                (
                    i,
                    cluster_silhouette_values.shape[0],
                    cluster_silhouette_values.mean(),
                    cluster_silhouette_values.min(),
                    cluster_silhouette_values.max(),
                )
            )
        silhouette_values = sorted(
            silhouette_values, key=lambda tup: tup[2], reverse=True
        )
        for s in silhouette_values:
            print(
                f"    Cluster {s[0]}: Size:{s[1]} | Avg:{s[2]:.2f} | Min:{s[3]:.2f} | Max: {s[4]:.2f}"
            )
    return km, km.labels_

##############################################################################################################


k = 19

clustering, cluster_labels = mbkmeans_clusters(X=corpus_embeddings, k=k, print_silhouette_values=True)

df_clusters = pd.DataFrame({
    "text": sentences,
    "cluster": cluster_labels
})

df_clusters.to_excel("SwissExcelResearchQ1.xlsx")

##############################################################################################################


df_test = pd.DataFrame({})

df_ref = pd.read_excel(r"D:\USERS-Load\PycharmProjects\pythonProject\CVS Files\Teachers prompt\Goldstandards.xlsx")

def_clusters = pd.read_excel(r"SwissExcelResearchQ1.xlsx")

listRef = df_ref['Gold standard']

listClusters = def_clusters['cluster']
listNewSentences = def_clusters['text']
listembReferences = []

for ref in listRef:
    listembReferences.append(model.encode(ref))

cosListRefToClusters = []
mm=0
for emb in corpus_embeddings:
    listMax = []
    for embref in listembReferences:
        cos_sim = util.cos_sim(emb, embref)
        listMax.append(cos_sim)
    index_max = np.argmax(listMax) #this is the right reference cluster
    cosListRefToClusters.append(index_max)
    df_test = df_test.append({
        "ID" : mm,
        "text": listNewSentences[mm],
        "cluster": listClusters[mm],
        "cos Sim": index_max
              },ignore_index=True)
    mm = mm+1


df_test.to_excel("SwissExcelResearchQ1.xlsx");


##############################################################################################################

tpCount= 0
tnCount  = 0

fnCount = 0
fpcount = 0


dfPairEvaluation = pd.read_excel(r"SwissExcelResearchQ1.xlsx")

iteration = dfPairEvaluation.count()['ID']

for i in range(0,iteration):
    for j in range (i+1, iteration):


        clusterSystem1 = dfPairEvaluation.query(f"ID == {i}")['cluster'].iloc[0]
        clusterGold1 = dfPairEvaluation.query(f"ID == {i}")['cos Sim'].iloc[0]
        clusterSystem2 = dfPairEvaluation.query(f"ID == {j}")['cluster'].iloc[0]
        clusterGold2 = dfPairEvaluation.query(f"ID == {j}")['cos Sim'].iloc[0]


        if (clusterSystem1 == clusterSystem2):

            if (clusterGold1 ==clusterGold2): # it is a true positive
                tpCount = tpCount+ 1

            else:
                fnCount = fnCount + 1   # it is a false negative
        else:
            if (clusterGold1 !=clusterGold2):
                tnCount = tnCount + 1  # it is a true negative
            else:
                fpcount = fpcount+1   # it is a false positive.

print("the value of tp is", tpCount)
print("the value of tn is", tnCount)
print("the value of fn is", fnCount)
print("the value of fb is", fpcount)

accuracy = (tpCount + tnCount)/(tpCount +tnCount + fpcount+ fnCount)

print("The accuracy is ",accuracy*100)  # the value is higher than the other accuracy.

percision = tpCount / (tpCount + fpcount)

print("The percision is ",percision*100)

recall = tpCount / (tpCount + fnCount)

print("The recall is ",recall*100)

##############################################################################################################

from collections import defaultdict
dfPurity = pd.read_excel(r"SwissExcelResearchQ1.xlsx")
d = defaultdict(list)

for j in range(0, k-1):
    mylist = dfPurity.query(f"cluster == {j}")['cos Sim']
    d[j].extend(mylist)

rightCounter = 0

for key, value in d.items():
    mostFreq = max(set(value), key = value.count)
    rightCounter = rightCounter + value.count(mostFreq)
    #print(key , value.count(mostFreq))

total = dfPurity.count()['ID']

print("purity = ", (rightCounter/total)*100)

Batches:   0%|          | 0/46 [00:00<?, ?it/s]

For n_clusters = 19
Silhouette coefficient: 0.02
Inertia:705.55859375
Silhouette values:
    Cluster 17: Size:29 | Avg:0.12 | Min:-0.01 | Max: 0.20
    Cluster 16: Size:88 | Avg:0.08 | Min:0.01 | Max: 0.19
    Cluster 1: Size:164 | Avg:0.06 | Min:-0.04 | Max: 0.15
    Cluster 5: Size:211 | Avg:0.05 | Min:-0.04 | Max: 0.13
    Cluster 10: Size:40 | Avg:0.04 | Min:-0.12 | Max: 0.13
    Cluster 6: Size:57 | Avg:0.01 | Min:-0.08 | Max: 0.09
    Cluster 14: Size:117 | Avg:0.01 | Min:-0.08 | Max: 0.09
    Cluster 13: Size:159 | Avg:0.01 | Min:-0.06 | Max: 0.09
    Cluster 11: Size:76 | Avg:0.00 | Min:-0.07 | Max: 0.06
    Cluster 12: Size:40 | Avg:0.00 | Min:-0.09 | Max: 0.09
    Cluster 0: Size:60 | Avg:-0.00 | Min:-0.10 | Max: 0.09
    Cluster 2: Size:138 | Avg:-0.01 | Min:-0.10 | Max: 0.07
    Cluster 3: Size:87 | Avg:-0.01 | Min:-0.10 | Max: 0.07
    Cluster 7: Size:27 | Avg:-0.01 | Min:-0.11 | Max: 0.05
    Cluster 15: Size:7 | Avg:-0.01 | Min:-0.05 | Max: 0.03
    Cluster 4: Size:30 | 

C:\Users\Ahmed\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:43: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  result = getattr(asarray(obj), method)(*args, **kwds)
C:\Users\Ahmed\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(a

KeyboardInterrupt: 

In [2]:
import os
from nltk import tokenize
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
import pandas as pd
from sentence_transformers import SentenceTransformer,util
import re


##############################################################################################################


model = SentenceTransformer('all-mpnet-base-v2', device="cuda")


##############################################################################################################


listStructureWords =  []

dfSrtucturingWords = pd.read_excel("../structureWords.xlsx")

listStructureWords = dfSrtucturingWords.iloc[:,0].tolist()

regex = r"\b(?:{})\b".format("|".join(listStructureWords))
sentences = []

df_fileName = pd.DataFrame({})

for filename in os.listdir(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\Only 100 - swiss Ads"):
   if (len(sentences) >= 1000):
        break
   with open(os.path.join(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\Only 100 - swiss Ads", filename)) as f:
       text = f.read()
       text = text.replace("ï»¿","")
       sents = re.split(regex, text)
       sents = tokenize.sent_tokenize(text)
       for s in sents:
           sentses = tokenize.sent_tokenize(s)
           if (s.isspace() or len(s) ==0):
               continue
           s = s.lower()
           if (len(s.split()) <5):
               continue
           for ss in sentses:
                 if (len(ss.split()) >5):
                    sentences.append(ss)
##############################################################################################################

corpus_embeddings = model.encode(sentences, show_progress_bar =True, device="cuda")

def mbkmeans_clusters(X, k, mb=500, print_silhouette_values=False):
    """Generate clusters.

    Args:
        X: Matrix of features.
        k: Number of clusters.
        mb: Size of mini-batches. Defaults to 500.
        print_silhouette_values: Print silhouette values per cluster.

    Returns:
        Trained clustering model and labels based on X.
    """
    km = MiniBatchKMeans(n_clusters=k, batch_size=mb).fit(X)
    print(f"For n_clusters = {k}")
    print(f"Silhouette coefficient: {silhouette_score(X, km.labels_):0.2f}")
    print(f"Inertia:{km.inertia_}")

    if print_silhouette_values:
        sample_silhouette_values = silhouette_samples(X, km.labels_)
        print(f"Silhouette values:")
        silhouette_values = []
        for i in range(k):
            cluster_silhouette_values = sample_silhouette_values[km.labels_ == i]
            silhouette_values.append(
                (
                    i,
                    cluster_silhouette_values.shape[0],
                    cluster_silhouette_values.mean(),
                    cluster_silhouette_values.min(),
                    cluster_silhouette_values.max(),
                )
            )
        silhouette_values = sorted(
            silhouette_values, key=lambda tup: tup[2], reverse=True
        )
        for s in silhouette_values:
            print(
                f"    Cluster {s[0]}: Size:{s[1]} | Avg:{s[2]:.2f} | Min:{s[3]:.2f} | Max: {s[4]:.2f}"
            )
    return km, km.labels_

##############################################################################################################


k = 19

clustering, cluster_labels = mbkmeans_clusters(X=corpus_embeddings, k=k, print_silhouette_values=True)

df_clusters = pd.DataFrame({
    "text": sentences,
    "cluster": cluster_labels
})

df_clusters.to_excel("SwissExcelResearchQ1-Normalised - AD.xlsx")

##############################################################################################################


df_test = pd.DataFrame({})

df_ref = pd.read_excel(r"D:\USERS-Load\PycharmProjects\pythonProject\CVS Files\Teachers prompt\Goldstandards.xlsx")

def_clusters = pd.read_excel(r"SwissExcelResearchQ1-Normalised - AD.xlsx")

listRef = df_ref['Gold standard']

listClusters = def_clusters['cluster']
listNewSentences = def_clusters['text']
listembReferences = []

for ref in listRef:
    listembReferences.append(model.encode(ref))

cosListRefToClusters = []
mm=0
for emb in corpus_embeddings:
    listMax = []
    for embref in listembReferences:
        cos_sim = util.cos_sim(emb, embref)
        listMax.append(cos_sim)
    index_max = np.argmax(listMax) #this is the right reference cluster
    cosListRefToClusters.append(index_max)
    df_test = df_test.append({
        "ID" : mm,
        "text": listNewSentences[mm],
        "cluster": listClusters[mm],
        "cos Sim": index_max
              },ignore_index=True)
    mm = mm+1


df_test.to_excel("SwissExcelResearchQ1-Normalised - AD.xlsx");



Batches:   0%|          | 0/32 [00:00<?, ?it/s]

For n_clusters = 19
Silhouette coefficient: 0.02
Inertia:529.9147338867188
Silhouette values:
    Cluster 17: Size:49 | Avg:0.12 | Min:0.05 | Max: 0.22
    Cluster 5: Size:49 | Avg:0.07 | Min:-0.06 | Max: 0.17
    Cluster 11: Size:39 | Avg:0.06 | Min:-0.02 | Max: 0.17
    Cluster 16: Size:62 | Avg:0.04 | Min:-0.05 | Max: 0.11
    Cluster 6: Size:79 | Avg:0.03 | Min:-0.10 | Max: 0.10
    Cluster 7: Size:41 | Avg:0.03 | Min:-0.02 | Max: 0.10
    Cluster 12: Size:56 | Avg:0.03 | Min:-0.02 | Max: 0.12
    Cluster 4: Size:66 | Avg:0.02 | Min:-0.08 | Max: 0.09
    Cluster 8: Size:48 | Avg:0.01 | Min:-0.07 | Max: 0.08
    Cluster 1: Size:54 | Avg:0.01 | Min:-0.07 | Max: 0.08
    Cluster 13: Size:42 | Avg:-0.00 | Min:-0.08 | Max: 0.08
    Cluster 2: Size:108 | Avg:-0.00 | Min:-0.10 | Max: 0.06
    Cluster 0: Size:32 | Avg:-0.01 | Min:-0.08 | Max: 0.04
    Cluster 15: Size:38 | Avg:-0.02 | Min:-0.07 | Max: 0.05
    Cluster 9: Size:73 | Avg:-0.02 | Min:-0.16 | Max: 0.05
    Cluster 3: Size:24 | 

C:\Users\Ahmed\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:43: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  result = getattr(asarray(obj), method)(*args, **kwds)
C:\Users\Ahmed\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(a